<a href="https://colab.research.google.com/github/insominiac21/Machine-Learning/blob/main/zip0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'skin-cancer-mnist-ham10000:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F54339%2F104884%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240627%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240627T093128Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D806ce1217cdda60b12c7a059f41119a0e99bb269a1aea2718ea5699af92e68768b8ef9a9ca552a54a74defa47046753867f7c705eb28c902c5b99c05691d725b067c726ac869aff5cdd82aee5c4cfdf40959a3708da2083adbccf665156ec43017b2dc16a75e2b7809465d227532a42d4853b2cb84c00bf0b67da0c70118fd00fac647f672f63069944901dd9ecefb4dc2f86169117084edcfb3b9de07c58dd8661e259f87d55ed386fccd2337e6280116241f87853a4d835de86e8295f0f40bda6709372f5e201fac6cd0669c01c61241778ad9c0dbfcd3c76081caa3ff73c668a9866630d81c84e1dc29735b360e6bd858021d52d9b2e73c0d4b1693183f0e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 5582914511 bytes downloaded
Downloaded and uncompressed: skin-cancer-mnist-ham10000
Data source import complete.


In [4]:
import os
import shutil
import pandas as pd
import zipfile

# Define the paths
metadata_path = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'  # Adjust this path if needed
images_path_1 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1'  # Adjust this path if needed
images_path_2 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2'  # Adjust this path if needed
output_dir = '/content/'  # Path to the output directory in Kaggle

# Create the output directories if they don't exist
labels = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']
for label in labels:
    os.makedirs(os.path.join(output_dir, label), exist_ok=True)

# Load the metadata
metadata = pd.read_csv(metadata_path)

# Function to copy images
def copy_image(source_dir, image_name, destination_dir):
    source = os.path.join(source_dir, image_name)
    destination = os.path.join(destination_dir, image_name)

    if os.path.exists(source):
        shutil.copy2(source, destination)
    else:
        return False
    return True

# Copy the images to the corresponding directories
for index, row in metadata.iterrows():
    image_name = row['image_id'] + '.jpg'
    label = row['dx']
    destination_dir = os.path.join(output_dir, label)

    # Try to copy from the first images directory
    if not copy_image(images_path_1, image_name, destination_dir):
        # If not found, try to copy from the second images directory
        if not copy_image(images_path_2, image_name, destination_dir):
            print(f"Image {image_name} not found in either source directory.")

# Zip the output directory
shutil.make_archive('/content/zip', 'zip', output_dir)

print("Images have been copied into directories based on their diagnosis labels and zipped.")

Images have been copied into directories based on their diagnosis labels and zipped.
